In [82]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [57]:
df_train=pd.read_csv('Train_Set_90621.csv')
df_test=pd.read_csv('Test_Set_90621.csv')

In [3]:
print(df_train.shape)
print(df_test.shape)

(195118, 29)
(83623, 28)


In [58]:
df_train.columns=df_train.columns.str.strip()
df_train.columns = df_train.columns.str.replace(" " ,"_")
df_test.columns=df_test.columns.str.strip()
df_test.columns = df_test.columns.str.replace(" " ,"_")

In [86]:
df_train.isnull().sum()

Application_ID                      0
Bank_Masked                        80
Bank_Type                          80
Approved_Timestamp                426
Name_Masked                         1
Business_Owner_State                3
Business_Industry_Type_Code         0
Approved_Year                       0
New_Business                       36
Term                                0
BankState                          81
Interest_Rate                       0
Employees                           0
Gross_Disbursed_Amount              0
Term_years                          0
Jobs_Retained                       0
Male_to_Female_Employees_Ratio      0
Expected_Company_Income             3
Funds_available_with_company        0
Gross_Apprv_Amount                  0
Company_Branch_Code                 0
City_or_Rural                       0
Jobs_Generated                      0
Carry-forward_Credit              978
Documents_Provided                552
Balance_Left                        0
Amount_Defau

In [5]:
df_train.head()

,Application_ID,Bank_Masked,Bank_Type,Approved_Timestamp,Name_Masked,Business_Owner_State,Business_Industry_Type_Code,Approved_Year,New_Business,Term,...,Gross_Apprv_Amount,Company_Branch_Code,City_or_Rural,Jobs_Generated,Carry-forward_Credit,Documents_Provided,Balance_Left,Amount_Defaulted,Final_Appved_Amount,Default_Status
0,1020200001,Bank_Name_316,Private,12-10-2005,Business_Name_10007,Uttar Pradesh,525161,2005,1.0,219,...,18550000,25650,1,3,N,N,0,0,13912500,0
1,1020200002,Bank_Name_1062,Govt,21-11-2006,Business_Name_130906,Uttar Pradesh,1231,2006,1.0,71,...,2450000,0,1,0,Y,N,0,2369500,1225000,1
2,1020200003,Bank_Name_39,Private,28-11-2007,Business_Name_105953,Uttar Pradesh,322442,2007,2.0,84,...,1750000,0,2,2,Y,N,0,0,1487500,0
3,1020200004,Bank_Name_3,Govt,18-04-2009,Business_Name_103197,Kerala,1231,2009,1.0,300,...,38850000,1,1,0,N,N,0,0,29137500,0
4,1020200005,Bank_Name_126,Govt,23-11-2002,Business_Name_118932,Delhi,1231,2002,2.0,153,...,2800000,1,2,0,N,Y,0,0,2380000,0


In [59]:
df_train_ob=df_train.select_dtypes(include=[np.object])
df_train_num=df_train.select_dtypes(include=[np.int64])

<ipython-input-59-2cd0638cfc13>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_train_ob=df_train.select_dtypes(include=[np.object])


In [22]:
df_train_ob['Documents_Provided'].value_counts()

N    170021
Y     24545
Name: Documents_Provided, dtype: int64

In [60]:
drop_cols=['Bank_Masked','Approved_Timestamp','Name_Masked','Business_Owner_State','BankState','Carry-forward_Credit']

In [61]:
df_train_ob=df_train_ob.drop(drop_cols,axis=1)

In [62]:
df_train_cat = pd.get_dummies(df_train_ob, drop_first=True)

In [63]:
train_in_cols=pd.concat([df_train_num,df_train_cat],axis=1)

In [64]:
train_in_cols=train_in_cols.drop(['Default_Status','Application_ID'],axis=1)
train_target_cols=df_train['Default_Status']

In [85]:
train_in_cols.isnull().sum()

Business_Industry_Type_Code       0
Approved_Year                     0
Term                              0
Interest_Rate                     0
Employees                         0
Gross_Disbursed_Amount            0
Term_years                        0
Jobs_Retained                     0
Male_to_Female_Employees_Ratio    0
Funds_available_with_company      0
Gross_Apprv_Amount                0
Company_Branch_Code               0
City_or_Rural                     0
Jobs_Generated                    0
Balance_Left                      0
Amount_Defaulted                  0
Final_Appved_Amount               0
Bank_Type_Private                 0
Documents_Provided_Y              0
dtype: int64

In [87]:
train_in_cols['Documents_Provided_Y'].value_counts()

0    170573
1     24545
Name: Documents_Provided_Y, dtype: int64

In [73]:
from sklearn.preprocessing import StandardScaler
standard=StandardScaler()
input_st=standard.fit_transform(train_in_cols)
input_st=pd.DataFrame(data=input_st,columns=train_in_cols.columns)

In [77]:
x_train,x_test,y_train,y_test=train_test_split(input_st,train_target_cols,test_size=0.3,random_state=1)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((136582, 19), (58536, 19), (136582,), (58536,))

In [81]:
dt_model=DecisionTreeRegressor(max_depth=5,random_state=1).fit(x_train,y_train)
validate_y_pred=dt_model.predict(x_test)
rmse=mean_squared_error(y_test,validate_y_pred,squared=False)
print('Root Mean Squared Error:',round(np.sqrt(rmse), 4))
r2_value_rfe = dt_model.score(x_train, y_train)
print('r-squared : ', r2_value_rfe)
r2_value_rfe_test = dt_model.score(x_test, y_test)
print('r-squared : ', r2_value_rfe_test)

Root Mean Squared Error: 0.4953
r-squared :  0.5949389929864912
r-squared :  0.5953295095374649


In [83]:
rf_model=RandomForestRegressor(max_depth=5,random_state=1).fit(x_train,y_train)
validate_y_pred=rf_model.predict(x_test)
rmse=mean_squared_error(y_test,validate_y_pred,squared=False)
print('Root Mean Squared Error:',round(np.sqrt(rmse), 4))
r2_value_rfe = rf_model.score(x_train, y_train)
print('r-squared : ', r2_value_rfe)
r2_value_rfe_test = rf_model.score(x_test, y_test)
print('r-squared : ', r2_value_rfe_test)

Root Mean Squared Error: 0.4947
r-squared :  0.5965054343865893
r-squared :  0.5969997192031772
